# Lung Disease Classification menggunakan Video Vision Transformer
<br>Nama : Husni Na'fa Mubarok
<br>NIM : 121450078

## Import Package

In [ ]:
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import librosa
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay, roc_curve, auc, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from itertools import combinations
from sklearn.preprocessing import LabelBinarizer
import random
import warnings
warnings.filterwarnings("ignore")

seed = 42

# Set seed for TensorFlow
tf.random.set_seed(seed)

# Set seed for NumPy
np.random.seed(seed)

# Set seed for Python's random module
random.seed(seed)

## EDA and Data Cleaning

### Import Dataframe & Processing

In [ ]:
train = pd.read_csv("/kaggle/input/airs-ai-in-respiratory-sounds/train.csv")

# Candidate IDs to remove
remove_ids = [
    '5ee582f2832c2',]
#     'd3ef8479c4212',
#     '912eb10321597',
#     'f0d3c857bec13',
#     '20d7c44ffabbb',
#     '2d1645a4f4db8'
# ]

# Filter out the unwanted candidateIDs
train = train[~train['candidateID'].isin(remove_ids)]

# Print info and null value summary
print(train.info())
print(train.isnull().sum())

# Display the updated dataframe
train

### Drop Null Column

In [ ]:
train = train.drop(columns=["coldPresent"])
train.isnull().sum()

### Sound Visualization & Processing

In [ ]:
# Define sound folder and candidate IDs
SOUND_FOLDER = "/kaggle/input/airs-ai-in-respiratory-sounds/sounds/sounds"
OUTPUT_FOLDER = "/kaggle/working/waveform_images"  # Folder to save images
os.makedirs(OUTPUT_FOLDER, exist_ok=True)  # Create the folder if it doesn't exist

candidate_ids = ["b87ea0dd760fa", "e000a41725f53", "253ba780a0398"]  # List of candidate IDs

# Generate the list of audio file paths
audio_paths = [os.path.join(SOUND_FOLDER, candidate_id, "cough.wav") for candidate_id in candidate_ids]

# Create subplots for multiple waveforms
fig, axes = plt.subplots(len(audio_paths), 1, figsize=(10, 4 * len(audio_paths)))

# Ensure axes is iterable for a single file
if len(audio_paths) == 1:
    axes = [axes]  

for i, file in enumerate(audio_paths):
    try:
        y, sr = librosa.load(file, sr=None)  # Load audio file
        librosa.display.waveshow(y, sr=sr, ax=axes[i])  # Plot waveform
        axes[i].set_title(f"Waveform dari {candidate_ids[i]}")  # Use candidate ID in title
        axes[i].set_xlabel("Waktu (s)")
        axes[i].set_ylabel("Amplitudo")
    except Exception as e:
        print(f"Error loading {file}: {e}")

# Save the figure
save_path = os.path.join(OUTPUT_FOLDER, "waveforms.png")
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"Waveform image saved to: {save_path}")

plt.show()

## Data Split & Pipeline

In [ ]:
SOUND_FOLDER = "/kaggle/input/airs-ai-in-respiratory-sounds/sounds/sounds"

data = train
N_MFCC = 20
BATCH_SIZE = 32
SEQ_LENGTH = 512
EMBED_DIM = 512
NUM_HEADS = 8
NUM_LAYERS = 6
NUM_FEATURE = len(data.columns) - 2
D_MODELS = EMBED_DIM
EPOCHS = 100

In [ ]:
def segment_cough_sound(signal, sr, cough_threshold=0.05, min_cough_duration=0.1, padding=0.05, visualize=True):
    hop_length = int(min_cough_duration * sr)
    if len(signal.shape) > 1:
        signal = np.mean(signal, axis=1)

    energy = librosa.feature.rms(y=signal, hop_length=hop_length)[0]

    # Normalize the energy values
    normalized_energy = (energy - np.min(energy)) / (np.max(energy) - np.min(energy))

    # Set the energy threshold for event detection
    energy_threshold = np.max(normalized_energy) * cough_threshold
    min_cough_samples = round(sr * min_cough_duration)

    # Find the cough segments
    cough_segments = []
    segments_time = []
    event_start = None

    for i, value in enumerate(normalized_energy):
        if value >= energy_threshold:
            if event_start is None:
                event_start = i * hop_length
        else:
            if event_start is not None:
                cough_duration = i * hop_length - event_start
                if cough_duration >= min_cough_samples:
                    event_end = i * hop_length + int(padding * sr)
                    event_start_pad = max(event_start - int(padding * sr), 0)
                    event_end = min(event_end, len(signal) - 1)
                    cough_segments.append(signal[event_start_pad: event_end + 1])
                    segments_time.append((event_start_pad / sr, event_end / sr))
                event_start = None

    if visualize:
        times = np.linspace(0, len(signal) / sr, num=len(signal))

        plt.figure(figsize=(14, 5))
        plt.plot(times, signal, label="Audio Signal")
        for start, end in segments_time:
            plt.axvspan(start, end, color='red', alpha=0.3, label="Cough Segment" if 'Cough Segment' not in plt.gca().get_legend_handles_labels()[1] else None)

        plt.title("Cough Sound Segmentation")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.legend(loc='upper right')
        plt.tight_layout()
        plt.show()

    return cough_segments

y, sr = librosa.load("/kaggle/input/airs-ai-in-respiratory-sounds/sounds/sounds/e000a41725f53/cough.wav")
visual_audio = segment_cough_sound(y, sr)

In [ ]:
def segment_cough_sound(signal, sr, cough_threshold=0.05, min_cough_duration=0.1, padding=0.05):

    hop_length = int(min_cough_duration*sr)
    if len(signal.shape) > 1:
        signal = np.mean(signal, axis=1)

    energy = librosa.feature.rms(y=signal, hop_length=hop_length)[0]

    # Normalize the energy values
    normalized_energy = (energy - np.min(energy)) / (np.max(energy) - np.min(energy))

    # Set the energy threshold for event detection
    cough_threshold = np.max(normalized_energy) * cough_threshold
    min_cough_samples = round(sr * min_cough_duration)


    # Find the cough segments
    cough_segments = []
    event_start = None

    for i, value in enumerate(normalized_energy):
        if value >= cough_threshold:
            if event_start is None:
                event_start = i*hop_length
        else:
            if event_start is not None:
                cough_duration = i*hop_length - event_start
                if cough_duration >= min_cough_samples:
                    event_end = i*hop_length + int(padding * sr)
                    event_start -= int(padding * sr)
                    event_start = max(event_start, 0)
                    cough_segments.append(signal[event_start: event_end+1])
                event_start = None

    # Convert cough segments to time in seconds
    # cough_segments = [(start / sr, end / sr) for start, end in cough_segments]

    return cough_segments

def extract_mfcc(file_path, n_mfcc=N_MFCC, target_length=SEQ_LENGTH):
    try:
        audio, sr = librosa.load(file_path)
        cough_segments = segment_cough_sound(audio, sr)

        # If no cough segments found, use full audio
        if not cough_segments:
            print(f"No cough segments detected for {file_path}, using full signal.")
            segmented_audio = audio
        else:
            segmented_audio = np.concatenate(cough_segments)

        # Extract MFCC from the segmented audio
        mfcc = librosa.feature.mfcc(y=segmented_audio, sr=sr, n_mfcc=n_mfcc)
        mfcc = mfcc.T  # shape: (time_steps, n_mfcc)

        # Pad or trim to target length
        if len(mfcc) > target_length:
            mfcc = mfcc[:target_length]
        elif len(mfcc) < target_length:
            pad_width = target_length - len(mfcc)
            mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')

        return tf.convert_to_tensor(mfcc, dtype=tf.float32)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return tf.zeros((target_length, n_mfcc), dtype=tf.float32)


def preprocess_features(row):
    age = row["age"] / 100.0
    pack_years = row["packYears"] / 100.0
    gender = row["gender"]
    tb_contact_history = row["tbContactHistory"]
    wheezing_history = row["wheezingHistory"]
    phlegm_cough = row["phlegmCough"]
    family_asthma_history = row["familyAsthmaHistory"]

    fever_history = row["feverHistory"]

    features = [
        age, gender, pack_years, tb_contact_history, wheezing_history,
        phlegm_cough, family_asthma_history, fever_history
    ]
    features = np.array(features).reshape(-1, 1)
    return features

def process_row(row):
    candidate_id = row["candidateID"]
    audio_path = os.path.join(SOUND_FOLDER, str(candidate_id), "cough.wav")
    mfcc = extract_mfcc(audio_path)
    features = preprocess_features(row)
    label = row["disease"]
    label = tf.one_hot(label, depth=3)
    return mfcc, features, label

def data_generator(data):
    for _, row in data.iterrows():
        mfcc, features, label = process_row(row)
        yield mfcc, features, label

def create_dataset(data, batch_size=BATCH_SIZE):
    output_signature = (
        tf.TensorSpec(shape=(SEQ_LENGTH, N_MFCC), dtype=tf.float32),
        tf.TensorSpec(shape=(NUM_FEATURE, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(3,), dtype=tf.int32),
    )
    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(data),
        output_signature=output_signature
    )
    return dataset.batch(batch_size).shuffle(256).prefetch(tf.data.AUTOTUNE)

def split_data(data, test_size=0.2, random_state=42):
    train_data, valid_data = train_test_split(
        data, test_size=test_size, random_state=random_state, stratify=data['disease']
    )
    return train_data, valid_data

train_data_df, valid_data_df = split_data(data)

train_dataset = create_dataset(train_data_df, batch_size=BATCH_SIZE)
valid_dataset = create_dataset(valid_data_df, batch_size=BATCH_SIZE)
train_dataset

In [ ]:
# # List of candidateIDs to check
# check_ids = [
#     '5ee582f2832c2',
#     'd3ef8479c4212',
#     '912eb10321597',
#     'f0d3c857bec13',
#     '20d7c44ffabbb',
#     '2d1645a4f4db8'
# ]

# # Check if any of these IDs are still in train_data_df
# remaining_ids = train_data_df[train_data_df['candidateID'].isin(check_ids)]

# # Display the result
# print(f"Remaining candidateIDs in train_data_df:\n{remaining_ids}")

## Transformer Component

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, embed_dim, trainable_embed=False, **kwargs):
        super().__init__(**kwargs)
        
        # Generate positional encoding matrix
        position_embedding_matrix = self.get_position_encoding(sequence_length, embed_dim)
        
        # Non-trainable Embedding layer to store positional encodings
        if trainable_embed == False:
            self.position_embedding_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=embed_dim,
                weights=[position_embedding_matrix],
                trainable=False
            )
        else:
            self.position_embedding_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=embed_dim,
                trainable=True
            )
             
    def get_position_encoding(self, seq_len, d, n=10000):
        P = np.zeros((seq_len, d))
        for k in range(seq_len):
            for i in np.arange(int(d/2)):
                denominator = np.power(n, 2*i/d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return P
 
    def call(self, inputs):        
        # Get position indices (0, 1, ..., sequence_length-1)
        sequence_length = tf.shape(inputs)[1]  # Ambil sequence_length dari inputs
        position_indices = tf.range(sequence_length)  # Shape: (sequence_length,)
        
        # Get positional encoding
        positional_encoding = self.position_embedding_layer(position_indices)  # Shape: (sequence_length, embed_dim)
        
        # Expand dimensions for broadcasting: (1, sequence_length, embed_dim)
        return positional_encoding[tf.newaxis, :, :]


class TransformerEncoderBlock(tf.keras.Model):
    def __init__(self, d_models, num_heads, num_layers=6, **kwargs):
        super().__init__(**kwargs)

        self.dense256 = layers.Dense(256, activation="relu")
        self.dense_transform = layers.Dense(d_models, activation="relu")
        self.num_layers = num_layers
        self.linear = layers.Dense(d_models, activation="relu")

        self.frame_positional_encoding = PositionalEncoding(
            sequence_length=NUM_FEATURE + 1, embed_dim=d_models, trainable_embed=True
        )
        self.patch_positional_encoding = PositionalEncoding(
            sequence_length=SEQ_LENGTH + 1, embed_dim=d_models, trainable_embed=True
        )

        # Define layers as lists
        self.attention_frame = [layers.MultiHeadAttention(num_heads=num_heads, key_dim=int(d_models/num_heads), dropout=0.2) 
                                for _ in range(num_layers)]
        self.attention_cross = [layers.MultiHeadAttention(num_heads=num_heads, key_dim=int(d_models/num_heads), dropout=0.2) 
                                for _ in range(num_layers)]
        self.layernorm_frame = [layers.LayerNormalization() for _ in range(num_layers)]
        self.layernorm_cross = [layers.LayerNormalization() for _ in range(num_layers)]
        self.dense_1 = [layers.Dense(d_models*2, activation="gelu") for _ in range(num_layers)]
        self.dense_2 = [layers.Dense(d_models) for _ in range(num_layers)]
        self.dropout = [layers.Dropout(0.2) for _ in range(num_layers)]

        # Learnable class tokens
        self.cls_token = self.add_weight(
            shape=(1, 1, d_models),
            initializer="random_normal",
            trainable=True,
            name="cls_token"
        )

        self.cls_token2 = self.add_weight(
            shape=(1, 1, d_models),
            initializer="random_normal",
            trainable=True,
            name="cls_token2"
        )

        # Final output layers
        self.out = layers.Dense(d_models, activation="gelu")
        self.out_drop = layers.Dropout(0.2)

    def call(self, inputs, training=False):
        """Transformer Encoder Forward Pass"""
        inputs, frame = inputs

        # Add class token
        batch_size = tf.shape(inputs)[0]
        inputs = self.dense_transform(self.dense256(inputs))
        cls_tokens = tf.broadcast_to(self.cls_token, [batch_size, 1, tf.shape(self.cls_token)[-1]])
        cls_tokens2 = tf.broadcast_to(self.cls_token2, [batch_size, 1, tf.shape(self.cls_token2)[-1]])

        # Process frame
        frame = self.linear(frame)
        frame = tf.concat([cls_tokens2, frame], axis=1)
        Zt = layers.Add()([frame, self.frame_positional_encoding(frame)])

        # Process inputs
        inputs = tf.concat([cls_tokens, inputs], axis=1)
        Zs = layers.Add()([inputs, self.patch_positional_encoding(inputs)])

        # Iterate over pre-defined layers (avoid Graph Mode error)
        for i in range(self.num_layers):
            Zt = self.layernorm_frame[i](Zt + self.attention_frame[i](query=Zt, value=Zt, key=Zt, training=training))
            Zts = tf.concat([Zs, Zt], axis=1)
            Zs = self.layernorm_cross[i](Zs + self.attention_cross[i](query=Zs, value=Zts, key=Zts, training=training))

            inputs = self.dense_1[i](Zs)
            inputs = self.dropout[i](inputs, training=training)
            inputs = self.dense_2[i](inputs)
            Zs += inputs

            inputsf = self.dense_1[i](Zt)
            inputsf = self.dropout[i](inputsf, training=training)
            inputsf = self.dense_2[i](inputsf)
            Zt += inputsf

        # Output
        Zt = Zt[:, 0, :]
        Zs = Zs[:, 0, :]
        out = tf.concat([Zs, Zt], axis=-1)
        out = self.out(out)
        out = self.out_drop(out)
        return out


    def create_look_ahead_mask(self, size):
        return tf.linalg.band_part(tf.ones((size, size)), -1, 0)


class Classifier(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.densel = layers.Dense(EMBED_DIM)
        self.out = layers.Dense(3, activation="softmax")

    def call(self, inputs):
        x = self.densel(inputs)
        x = self.out(x)
        return x



class MainModel(keras.Model):
    def __init__(self, encoder, classifier, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.classifier = classifier
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")

    def call(self, inputs, training=False):
        """Forward pass through encoder and classifier"""
        encoder_out = self.encoder(inputs, training=training)
        preds = self.classifier(encoder_out)
        return preds  # Returns predicted probabilities

    def calculate_loss(self, y_pred, y_true):
        return self.loss(y_true, y_pred)

    def calculate_acc(self, y_pred, y_true):
        predicted_classes = tf.argmax(y_pred, axis=1)
        y_true = tf.argmax(y_true, axis=1)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(y_true, predicted_classes), dtype=tf.float32))
        return accuracy

    def train_step(self, batch_data):
        batch, batch2, label = batch_data

        with tf.GradientTape() as tape:
            preds = self(inputs=[batch, batch2], training=True)
            batch_loss = self.calculate_loss(preds, label)
            batch_acc = self.calculate_acc(preds, label)

        train_vars = self.trainable_variables
        grads = tape.gradient(batch_loss, train_vars)
        self.optimizer.apply_gradients(zip(grads, train_vars))

        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        return {"loss": self.loss_tracker.result(), "accuracy": self.acc_tracker.result()}

    def test_step(self, batch_data):
        batch, batch2, label = batch_data
        preds = self(inputs=[batch, batch2], training=False)
        batch_loss = self.calculate_loss(preds, label)
        batch_acc = self.calculate_acc(preds, label)

        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        return {"loss": self.loss_tracker.result(), "accuracy": self.acc_tracker.result()}

    @property
    def metrics(self):
        return [self.loss_tracker, self.acc_tracker]

## Modeling

In [ ]:
encoder = TransformerEncoderBlock(d_models=D_MODELS, num_heads=NUM_HEADS, num_layers=NUM_LAYERS)
classifier = Classifier()
model = MainModel(encoder, classifier)

## Training

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super().__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {
            "d_model": self.d_model.numpy(),  # Konversi ke tipe Python agar bisa diserialisasi
            "warmup_steps": self.warmup_steps
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
# Define the loss function
cross_entropy = keras.losses.CategoricalCrossentropy()

# EarlyStopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor = "val_loss", mode='min', patience=15, restore_best_weights=True)
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0000001)
# Optimizer
lr = CustomSchedule(D_MODELS)
# optimizer = tf.keras.optimizers.AdamW(
#     learning_rate=0.001,
#     weight_decay=0.004,
#     beta_1=0.9,
#     beta_2=0.999,
#     epsilon=1e-07,
# )
optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr,
    beta_1=0.9,
    beta_2=0.98,
    weight_decay=1e-5,
    epsilon=1e-9
)

# Compile the model
model.compile(optimizer=optimizer, loss=cross_entropy)

In [ ]:
model.summary()

In [ ]:
#Fit the model
history = model.fit(train_dataset, epochs=EPOCHS, validation_data=valid_dataset, callbacks=[early_stopping])

### Visualize Training Accuracy & Loss

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 8))
plt.plot(history.history['accuracy'], marker='o', linestyle='-')
plt.plot(history.history['val_accuracy'], marker='o', linestyle='--')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.grid(True)

# Save the accuracy plot
plt.savefig("training_validation_accuracy.png", dpi=300, bbox_inches='tight')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(12, 8))
plt.plot(history.history['loss'], marker='o', linestyle='-')
plt.plot(history.history['val_loss'], marker='o', linestyle='--')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.grid(True)

# Save the loss plot
plt.savefig("training_validation_loss.png", dpi=300, bbox_inches='tight')
plt.show()

### Save Model

In [ ]:
# Save the entire model as a `.keras` zip archive.
model.save('big_model_lung_disease_classification_transformer.keras')

In [ ]:
model.save_weights("model_weights.weights.h5")

## Model Evaluation

In [ ]:
# def extract_mfcc(file_path, n_mfcc=N_MFCC, target_length=SEQ_LENGTH):
#     try:
#         audio, sr = librosa.load(file_path)
#         mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
#         mfcc = mfcc.T
#         if len(mfcc) > target_length:
#             mfcc = mfcc[:target_length]
#         elif len(mfcc) < target_length:
#             pad_width = target_length - len(mfcc)
#             mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')
#         return mfcc
#     except Exception as e:
#         print(f"Error processing {file_path}: {e}")

def extract_mfcc(file_path, n_mfcc=N_MFCC, target_length=SEQ_LENGTH):
    try:
        audio, sr = librosa.load(file_path)
        cough_segments = segment_cough_sound(audio, sr)

        # If no cough segments found, use full audio
        if not cough_segments:
            print(f"No cough segments detected for {file_path}, using full signal.")
            segmented_audio = audio
        else:
            segmented_audio = np.concatenate(cough_segments)

        # Extract MFCC from the segmented audio
        mfcc = librosa.feature.mfcc(y=segmented_audio, sr=sr, n_mfcc=n_mfcc)
        mfcc = mfcc.T  # shape: (time_steps, n_mfcc)

        # Pad or trim to target length
        if len(mfcc) > target_length:
            mfcc = mfcc[:target_length]
        elif len(mfcc) < target_length:
            pad_width = target_length - len(mfcc)
            mfcc = np.pad(mfcc, ((0, pad_width), (0, 0)), mode='constant')

        return tf.convert_to_tensor(mfcc, dtype=tf.float32)

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return tf.zeros((target_length, n_mfcc), dtype=tf.float32)

def preprocess_features(row):
    age = row["age"] / 100.0
    pack_years = row["packYears"] / 100.0
    gender = row["gender"]
    tb_contact_history = row["tbContactHistory"]
    wheezing_history = row["wheezingHistory"]
    phlegm_cough = row["phlegmCough"]
    family_asthma_history = row["familyAsthmaHistory"]

    fever_history = row["feverHistory"]

    features = [
        age, gender, pack_years, tb_contact_history, wheezing_history,
        phlegm_cough, family_asthma_history, fever_history
    ]
    features = np.array(features).reshape(-1, 1)
    return features

def process_row(row):
    candidate_id = row["candidateID"]
    audio_path = os.path.join(SOUND_FOLDER, str(candidate_id), "cough.wav")
    mfcc = extract_mfcc(audio_path)
    features = preprocess_features(row)
    mfcc = tf.convert_to_tensor(mfcc, dtype=tf.float32)
    features = tf.convert_to_tensor(features, dtype=tf.float32)
    mfcc.set_shape((SEQ_LENGTH, N_MFCC))
    features.set_shape((NUM_FEATURE, 1))
    mfcc = tf.expand_dims(mfcc, axis=0)
    features = tf.expand_dims(features, axis=0)
    return mfcc, features

def predict(model, mfcc, features):
    encoder_out = model.encoder([mfcc, features], training=False)
    pred = model.classifier(encoder_out)
    predicted_classes = tf.argmax(pred, axis=1)
    return int(predicted_classes[0]), pred


In [ ]:
valid_dataset_auc = valid_data_df.copy()
valid_dataset_auc = valid_dataset_auc.drop(columns=["disease"])
valid_dataset_dict = valid_dataset_auc.to_dict(orient = 'records')
# List untuk menyimpan hasil prediksi
results = []

for row in valid_dataset_dict:
    mfcc, features = process_row(row)  # Proses data
    prediction = predict(model, mfcc, features)  # Prediksi

    # Tambahkan hasil ke list
    results.append({
        "candidateID": row["candidateID"],
        "y_score": prediction[1].numpy().tolist(),
        "disease": prediction[0]
    })

# Konversi hasil ke DataFrame
y_pred_prob = pd.DataFrame(results)

In [ ]:
y_test = valid_data_df['disease']
y_score = y_pred_prob['y_score']
y_score = np.asarray(y_score)
y_score = np.array([item[0] for item in y_score])

### Confusion matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred_prob['disease'], labels=[0, 1, 2])

# Display it
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1, 2])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - 3 Class")
plt.show()
plt.savefig('Confusion Matrix - 3 Class.png', dpi=300, bbox_inches='tight')  

### Accuracy, Precision, Recall, F1 Score

In [ ]:
accuracy = accuracy_score(y_test, y_pred_prob['disease'])
precision = precision_score(y_test, y_pred_prob['disease'], average='macro')
recall = recall_score(y_test, y_pred_prob['disease'], average='macro')
f1 = f1_score(y_test, y_pred_prob['disease'], average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

### ROC Curve

In [ ]:
pair_list = list(combinations(np.unique(y_pred_prob['disease']), 2))
print(pair_list)

In [ ]:
fpr_grid = np.linspace(0.0, 1.0, 1000)

In [ ]:
label_binarizer = LabelBinarizer().fit(train_data_df['disease'])

pair_scores = []
mean_tpr = dict()

# Make sure the output directory exists
os.makedirs("roc_plots", exist_ok=True)

for ix, (label_a, label_b) in enumerate(pair_list):
    a_mask = y_test == label_a
    b_mask = y_test == label_b
    ab_mask = np.logical_or(a_mask, b_mask)

    a_true = a_mask[ab_mask]
    b_true = b_mask[ab_mask]

    idx_a = np.flatnonzero(label_binarizer.classes_ == label_a)[0]
    idx_b = np.flatnonzero(label_binarizer.classes_ == label_b)[0]

    fpr_a, tpr_a, _ = roc_curve(a_true, y_score[ab_mask, idx_a])
    fpr_b, tpr_b, _ = roc_curve(b_true, y_score[ab_mask, idx_b])

    mean_tpr[ix] = np.zeros_like(fpr_grid)
    mean_tpr[ix] += np.interp(fpr_grid, fpr_a, tpr_a)
    mean_tpr[ix] += np.interp(fpr_grid, fpr_b, tpr_b)
    mean_tpr[ix] /= 2
    mean_score = auc(fpr_grid, mean_tpr[ix])
    pair_scores.append(mean_score)

    fig, ax = plt.subplots(figsize=(6, 6))
    plt.plot(
        fpr_grid,
        mean_tpr[ix],
        label=f"Mean {label_a} vs {label_b} (AUC = {mean_score :.2f})",
        linestyle=":",
        linewidth=4,
    )
    RocCurveDisplay.from_predictions(
        a_true,
        y_score[ab_mask, idx_a],
        ax=ax,
        name=f"{label_a} as positive class",
    )
    RocCurveDisplay.from_predictions(
        b_true,
        y_score[ab_mask, idx_b],
        ax=ax,
        name=f"{label_b} as positive class",
    )
    ax.set(
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"{label_a} vs {label_b} ROC curves",
    )
    ax.legend(loc="lower right")

    # Save the figure
    filename = f"roc_plots/roc_{label_a}_vs_{label_b}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    plt.close(fig)  # Close to free memory

## Predict New Data

### Data Processing

In [ ]:
test = pd.read_csv("/kaggle/input/airs-ai-in-respiratory-sounds/test.csv")
print(test.info())
print(test.isnull().sum())
test

In [ ]:
test = test.drop(columns=["coldPresent"])
test.isnull().sum()

In [ ]:
#new_data_dataset = new_data_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
# Recreate the model instance (architecture must match)
#model = MainModel(encoder=TransformerEncoderBlock(d_models=EMBED_DIM, num_heads=NUM_HEADS), classifier=Classifier())

# Load the saved weights
#model.load_weights("/kaggle/working/model_weights.weights.h5")

In [ ]:
test_dict = test.to_dict(orient="records")

### Prediksi data baru

In [ ]:
# List untuk menyimpan hasil prediksi
results = []

for row in test_dict:
    mfcc, features = process_row(row)  # Proses data
    prediction = predict(model, mfcc, features)  # Prediksi

    # Tambahkan hasil ke list
    results.append({
        "candidateID": row["candidateID"],
        "disease": prediction[0]
    })

# Konversi hasil ke DataFrame
results_df = pd.DataFrame(results)

# Simpan ke CSV
output_path = "/kaggle/working/predictions.csv"
results_df.to_csv(output_path, index=False)

print(f"Hasil prediksi disimpan di: {output_path}")